##### Copyright 2020 Die TensorFlow-Autoren.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Einführung in Gradienten und automatische Differenzierung

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Auf TensorFlow.org anzeigen</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">In Google Colab ausführen</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Notizbuch herunterladen</a></td>
</table>

## Automatische Differenzierung und Gradienten

[Die automatische Differenzierung](https://en.wikipedia.org/wiki/Automatic_differentiation) ist nützlich für die Implementierung von maschinellen Lernalgorithmen wie [Backpropagation](https://en.wikipedia.org/wiki/Backpropagation) zum Trainieren neuronaler Netze.

In diesem Leitfaden werden wir diskutieren, wie Sie Gradienten mit TensorFlow berechnen können, insbesondere bei der eifrigen Ausführung.

## Aufstellen

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

## Gradienten berechnen

Um automatisch zu differenzieren, muss sich TensorFlow merken, welche Operationen in welcher Reihenfolge während des *Vorwärtsdurchgangs* ausgeführt werden. Während des *Rückwärtsdurchgangs* durchläuft TensorFlow dann diese Liste von Vorgängen in umgekehrter Reihenfolge, um Gradienten zu berechnen.

## Verlaufsbänder

TensorFlow stellt die [tf.GradientTape-](https://www.tensorflow.org/api_docs/python/tf/GradientTape) API zur automatischen Differenzierung bereit; das heißt, den Gradienten einer Berechnung in Bezug auf einige Eingaben zu berechnen, normalerweise `tf.Variable` s. TensorFlow „zeichnet“ relevante Operationen, die im Kontext eines `tf.GradientTape` ausgeführt werden, auf einem „Band“ auf. TensorFlow verwendet dann dieses Band, um die Gradienten einer „aufgezeichneten“ Berechnung unter Verwendung der [Rückwärtsmodus-Differenzierung](https://en.wikipedia.org/wiki/Automatic_differentiation) zu berechnen.

Hier ist ein einfaches Beispiel:

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2

Nachdem Sie einige Operationen aufgezeichnet haben, verwenden Sie `GradientTape.gradient(target, sources)` , um den Gradienten eines Ziels (oft ein Verlust) relativ zu einer Quelle (oft die Variablen des Modells) zu berechnen.

In [ ]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

Das obige Beispiel verwendet Skalare, aber `tf.GradientTape` funktioniert genauso einfach mit jedem Tensor:

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

Um den Gradienten von `y` in Bezug auf beide Variablen zu erhalten, können Sie beide als Quellen an die `gradient` übergeben. Das Band ist flexibel in der Art und Weise, wie Quellen übergeben werden, und akzeptiert jede verschachtelte Kombination von Listen oder Wörterbüchern und gibt den Gradienten auf die gleiche Weise strukturiert zurück (siehe `tf.nest` ).

In [ ]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

Der Gradient in Bezug auf jede Quelle hat die Form der Quelle:

In [ ]:
print(w.shape)
print(dl_dw.shape)

Hier ist noch einmal die Gradientenberechnung, diesmal mit einem Wörterbuch von Variablen:

In [ ]:
my_vars = {
    'w': tf.Variable(tf.random.normal((3, 2)), name='w'),
    'b': tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
}

grad = tape.gradient(loss, my_vars)
grad['b']

## Gradienten in Bezug auf ein Modell

Es ist üblich, `tf.Variables` in einem `tf.Module` oder einer seiner Unterklassen ( `layers.Layer` , `keras.Model` ) für [Checkpointing](checkpoint.ipynb) und [Export](saved_model.ipynb) zu sammeln.

In den meisten Fällen möchten Sie Gradienten in Bezug auf die trainierbaren Variablen eines Modells berechnen. Da alle Unterklassen von `tf.Module` ihre Variablen in der Eigenschaft `Module.trainable_variables` aggregieren, können Sie diese Gradienten in wenigen Codezeilen berechnen: 

In [ ]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)

In [ ]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape: {g.shape}')

<a id="watches"></a>

## Steuern, was das Band sieht

Das Standardverhalten besteht darin, alle Vorgänge nach dem Zugriff auf eine trainierbare `tf.Variable` . Die Gründe dafür sind:

- Das Band muss wissen, welche Operationen im Vorwärtsdurchlauf aufzuzeichnen sind, um die Gradienten im Rückwärtsdurchlauf zu berechnen.
- Das Band enthält Verweise auf Zwischenausgänge, sodass Sie keine unnötigen Vorgänge aufzeichnen möchten.
- Der häufigste Anwendungsfall ist die Berechnung des Verlustgradienten in Bezug auf alle trainierbaren Variablen eines Modells.

Beispielsweise kann im Folgenden kein Gradient berechnet werden, da der `tf.Tensor` standardmäßig nicht "beobachtet" wird und die `tf.Variable` nicht trainierbar ist:

In [ ]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

Sie können die vom Band überwachten Variablen mit der Methode `GradientTape.watched_variables` auflisten:

In [ ]:
[var.name for var in tape.watched_variables()]

`tf.GradientTape` bietet Hooks, die dem Benutzer die Kontrolle darüber geben, was angesehen wird oder was nicht.

Um Gradienten in Bezug auf einen `tf.Tensor` , müssen Sie `GradientTape.watch(x)` aufrufen:

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x**2

# dy = 2x * dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

Um das Standardverhalten des Überwachens aller `tf.Variables` zu deaktivieren, legen Sie beim Erstellen des Verlaufsbands `watch_accessed_variables=False` . Diese Berechnung verwendet zwei Variablen, verbindet aber nur den Gradienten für eine der Variablen:

In [ ]:
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

Da `GradientTape.watch` nicht auf `x0` , wird in Bezug darauf kein Gradient berechnet:

In [ ]:
# dy = 2x * dx
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print('dy/dx0:', grad['x0'])
print('dy/dx1:', grad['x1'].numpy())

## Zwischenergebnisse

Sie können auch Gradienten der Ausgabe in Bezug auf Zwischenwerte anfordern, die innerhalb des `tf.GradientTape` -Kontexts berechnet wurden.

In [ ]:
x = tf.constant(3.0)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = x * x
  z = y * y

# Use the tape to compute the gradient of z with respect to the
# intermediate value y.
# dz_dx = 2 * y, where y = x ** 2
print(tape.gradient(z, y).numpy())

Standardmäßig werden die von einem `GradientTape` gehaltenen Ressourcen freigegeben, sobald die Methode `GradientTape.gradient()` aufgerufen wird. Um mehrere Farbverläufe über dieselbe Berechnung zu berechnen, erstellen Sie ein `persistent` Farbverlaufsband. Dies ermöglicht mehrere Aufrufe der `gradient()` Methode, da Ressourcen freigegeben werden, wenn das Tape-Objekt der Garbage Collection unterzogen wird. Zum Beispiel:

In [ ]:
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy())  # 108.0 (4 * x**3 at x = 3)
print(tape.gradient(y, x).numpy())  # 6.0 (2 * x)

In [ ]:
del tape   # Drop the reference to the tape

## Hinweise zur Leistung

- Es gibt einen kleinen Overhead, der mit der Durchführung von Operationen innerhalb eines Verlaufsbandkontexts verbunden ist. Für die eifrigste Ausführung ist dies kein merklicher Kostenfaktor, aber Sie sollten dennoch Bandkontext nur in den Bereichen verwenden, in denen dies erforderlich ist.

- Gradientenbänder verwenden Speicher zum Speichern von Zwischenergebnissen, einschließlich Eingaben und Ausgaben, zur Verwendung während des Rückwärtsdurchlaufs.

    Aus Effizienzgründen müssen einige Operationen (wie `ReLU` ) ihre Zwischenergebnisse nicht behalten und werden während des Vorwärtsdurchgangs gekürzt. Wenn Sie jedoch auf Ihrem Band `persistent=True` verwenden, wird *nichts verworfen* und Ihre maximale Speicherauslastung wird höher sein.

## Gradienten von nicht skalaren Zielen

Ein Gradient ist im Grunde eine Operation auf einem Skalar.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient(y0, x).numpy())
print(tape.gradient(y1, x).numpy())

Wenn Sie also nach dem Gradienten mehrerer Ziele fragen, lautet das Ergebnis für jede Quelle:

- Der Gradient der Summe der Ziele oder gleichwertig
- Die Summe der Gradienten jedes Ziels.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient({'y0': y0, 'y1': y1}, x).numpy())

Wenn das/die Ziel(e) nicht skalar sind, wird in ähnlicher Weise der Gradient der Summe berechnet:

In [ ]:
x = tf.Variable(2.)

with tf.GradientTape() as tape:
  y = x * [3., 4.]

print(tape.gradient(y, x).numpy())

Dies macht es einfach, den Gradienten der Summe einer Sammlung von Verlusten oder den Gradienten der Summe einer elementweisen Verlustberechnung zu nehmen.

Wenn Sie für jedes Element einen separaten Farbverlauf benötigen, siehe [Jacobians](advanced_autodiff.ipynb#jacobians) .

In einigen Fällen können Sie den Jacobi überspringen. Bei einer elementweisen Berechnung ergibt der Gradient der Summe die Ableitung jedes Elements in Bezug auf sein Eingangselement, da jedes Element unabhängig ist:

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.nn.sigmoid(x)

dy_dx = tape.gradient(y, x)

In [ ]:
plt.plot(x, y, label='y')
plt.plot(x, dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

## Kontrollfluss

Da Bänder Vorgänge während ihrer Ausführung aufzeichnen, wird der Ablauf der Python-Steuerung (z. B. unter Verwendung von `if` s und `while` s) natürlich gehandhabt.

Hier wird für jeden Zweig eines `if` eine andere Variable verwendet. Der Gradient verbindet sich nur mit der verwendeten Variable:

In [ ]:
x = tf.constant(1.0)

v0 = tf.Variable(2.0)
v1 = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  if x > 0.0:
    result = v0
  else:
    result = v1**2 

dv0, dv1 = tape.gradient(result, [v0, v1])

print(dv0)
print(dv1)

Denken Sie nur daran, dass die Steueranweisungen selbst nicht differenzierbar sind, sodass sie für Gradienten-basierte Optimierer unsichtbar sind.

Abhängig vom Wert von `x` im obigen Beispiel zeichnet das Band entweder `result = v0` oder `result = v1**2` auf. Der Gradient bezüglich `x` ist immer `None` .

In [ ]:
dx = tape.gradient(result, x)

print(dx)

## Einen Gradienten von `None`

Wenn ein Ziel nicht mit einer Quelle verbunden ist, erhalten Sie einen Gradienten von `None` .


In [ ]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y * y
print(tape.gradient(z, x))

Hier ist `z` offensichtlich nicht mit `x` verbunden, aber es gibt mehrere weniger offensichtliche Möglichkeiten, wie ein Gradient getrennt werden kann.

### 1. Eine Variable durch einen Tensor ersetzt.

Im Abschnitt ["Steuern, was das Band überwacht"](#watches) haben Sie gesehen, dass das Band automatisch eine `tf.Variable` , aber keinen `tf.Tensor` .

Ein häufiger Fehler besteht darin, versehentlich eine `tf.Variable` durch eine `tf.Tensor` zu ersetzen, anstatt `Variable.assign` zu verwenden, um die `tf.Variable` zu aktualisieren. Hier ist ein Beispiel:

In [ ]:
x = tf.Variable(2.0)

for epoch in range(2):
  with tf.GradientTape() as tape:
    y = x+1

  print(type(x).__name__, ":", tape.gradient(y, x))
  x = x + 1   # This should be `x.assign_add(1)`

### 2. Berechnungen außerhalb von TensorFlow durchgeführt

Das Band kann den Gradientenpfad nicht aufzeichnen, wenn die Berechnung TensorFlow beendet. Zum Beispiel:

In [ ]:
x = tf.Variable([[1.0, 2.0],
                 [3.0, 4.0]], dtype=tf.float32)

with tf.GradientTape() as tape:
  x2 = x**2

  # This step is calculated with NumPy
  y = np.mean(x2, axis=0)

  # Like most ops, reduce_mean will cast the NumPy array to a constant tensor
  # using `tf.convert_to_tensor`.
  y = tf.reduce_mean(y, axis=0)

print(tape.gradient(y, x))

### 3. Nahm Gradienten durch eine ganze Zahl oder einen String

Ganzzahlen und Strings sind nicht differenzierbar. Wenn ein Berechnungspfad diese Datentypen verwendet, gibt es keinen Gradienten.

Niemand erwartet, dass Strings differenzierbar sind, aber es ist leicht, versehentlich eine `int` -Konstante oder -Variable zu erstellen, wenn Sie den `dtype` nicht angeben.

In [ ]:
# The x0 variable has an `int` dtype.
x = tf.Variable([[2, 2],
                 [2, 2]])

with tf.GradientTape() as tape:
  # The path to x1 is blocked by the `int` dtype here.
  y = tf.cast(x, tf.float32)
  y = tf.reduce_sum(x)

print(tape.gradient(y, x))

TensorFlow konvertiert nicht automatisch zwischen Typen, sodass Sie in der Praxis häufig einen Typfehler anstelle eines fehlenden Gradienten erhalten.

### 5. Farbverläufe durch ein zustandsbehaftetes Objekt genommen

Zustand stoppt Gradienten. Wenn Sie von einem zustandsbehafteten Objekt lesen, kann das Band nur den aktuellen Zustand sehen, nicht die Historie, die dazu geführt hat.

Ein `tf.Tensor` ist unveränderlich. Sie können einen einmal erstellten Tensor nicht mehr ändern. Es hat einen *Wert* , aber keinen *Zustand* . Alle bisher besprochenen Operationen sind außerdem zustandslos: Die Ausgabe einer `tf.matmul` hängt nur von ihren Eingaben ab.

Eine `tf.Variable` hat einen internen Zustand, ihren Wert. Wenn Sie die Variable verwenden, wird der Status gelesen. Es ist normal, einen Gradienten in Bezug auf eine Variable zu berechnen, aber der Status der Variablen verhindert, dass Gradientenberechnungen weiter zurückgehen. Zum Beispiel:


In [ ]:
x0 = tf.Variable(3.0)
x1 = tf.Variable(0.0)

with tf.GradientTape() as tape:
  # Update x1 = x1 + x0.
  x1.assign_add(x0)
  # The tape starts recording from x1.
  y = x1**2   # y = (x1 + x0)**2

# This doesn't work.
print(tape.gradient(y, x0))   #dy/dx0 = 2*(x1 + x2)

In ähnlicher Weise sind `tf.data.Dataset` Iteratoren und `tf.queue` s zustandsbehaftet und stoppen alle Gradienten auf Tensoren, die sie durchlaufen.

## Keine Steigung registriert

Einige `tf.Operation` s sind **als nicht differenzierbar registriert** und geben `None` zurück. Andere haben **kein Gefälle registriert** .

Die Seite [tf.raw_ops](https://www.tensorflow.org/api_docs/python/tf/raw_ops) zeigt, bei welchen Low-Level-Operationen Gradienten registriert sind.

Wenn Sie versuchen, einen Gradienten durch eine Gleitkommaoperation zu nehmen, für die kein Gradient registriert ist, gibt das Band einen Fehler aus, anstatt stillschweigend `None` zurückzugeben. So wissen Sie, dass etwas schief gelaufen ist.

Zum Beispiel umschließt die Funktion `tf.image.adjust_contrast` [raw_ops.AdjustContrastv2](https://www.tensorflow.org/api_docs/python/tf/raw_ops#.AdjustContrastv2) , die einen Farbverlauf haben könnte, aber der Farbverlauf ist nicht implementiert:


In [ ]:
image = tf.Variable([[[0.5, 0.0, 0.0]]])
delta = tf.Variable(0.1)

with tf.GradientTape() as tape:
  new_image = tf.image.adjust_contrast(image, delta)

try:
  print(tape.gradient(new_image, [image, delta]))
  assert False   # This should not happen.
except LookupError as e:
  print(f'{type(e).__name__}: {e}')


Wenn Sie diese Operation unterscheiden müssen, müssen Sie entweder den Farbverlauf implementieren und registrieren (mit `tf.RegisterGradient` ) oder die Funktion mit anderen Operationen neu implementieren.

## Nullen statt Keine

In manchen Fällen wäre es bequemer, 0 statt `None` für unverbundene Farbverläufe zu erhalten. Mit dem Argument `unconnected_gradients` können Sie entscheiden, was zurückgegeben werden soll, wenn Sie nicht verbundene Farbverläufe haben:

In [ ]:
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))